In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie = pd.read_csv('movies.csv')

In [3]:
movie.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movie.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [5]:
movie.shape

(4803, 24)

In [6]:
sel_feats = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(sel_feats)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
for feat in sel_feats:
    movie[feat] = movie[feat].fillna('')

In [8]:
combined_feats = movie['genres'] + ' '+ movie['keywords'] +' '+ movie['tagline'] +' '+ movie['cast'] +' '+ movie['director']

In [9]:
combined_feats

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
feat_vects = vectorizer.fit_transform(combined_feats)

In [12]:
print(feat_vects)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [13]:
# getting similarity confidence values
similarity = cosine_similarity(feat_vects)
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [14]:
similarity.shape

(4803, 4803)

In [15]:
titles = movie['title'].to_list()
titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [16]:
movie_name = input('Enter your favourite movie name: ')

Enter your favourite movie name: amazing spiderman


In [17]:

# finding close match of the movie name entered by the user
close_match = difflib.get_close_matches(movie_name, titles)
close_match

['The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Saving Silverman']

In [18]:
close_match = close_match[0]
close_match

'The Amazing Spider-Man'

In [19]:
movie_index = movie[movie.title == close_match]['index'].values[0]
movie_index

20

In [20]:
titles[159]

'Spider-Man'

In [21]:
#getting similarity scores
similarity_score = list(enumerate(similarity[movie_index]))
similarity_score

[(0, 0.03971480215415495),
 (1, 0.08780812424608472),
 (2, 0.014586513752027637),
 (3, 0.012468240369946643),
 (4, 0.041694129432257705),
 (5, 0.04114142570617925),
 (6, 0.00820295531428505),
 (7, 0.11828290690100095),
 (8, 0.06092182196852157),
 (9, 0.1219297197648043),
 (10, 0.03549747692977713),
 (11, 0.04420267417088956),
 (12, 0.027104396289540734),
 (13, 0.017765277287414463),
 (14, 0.06322007762018508),
 (15, 0.04673948821778939),
 (16, 0.11215733913329524),
 (17, 0.025928501631657157),
 (18, 0.039747210481332416),
 (19, 0.03878893356718093),
 (20, 1.0),
 (21, 0.026730880550239217),
 (22, 0.03031236054099938),
 (23, 0.06388985697827622),
 (24, 0.030604210293548685),
 (25, 0.0),
 (26, 0.0886639035417785),
 (27, 0.04907579366134701),
 (28, 0.01927320401500121),
 (29, 0.01491644649584264),
 (30, 0.11460131343801569),
 (31, 0.09349691478373992),
 (32, 0.030236727134394875),
 (33, 0.11754797974032946),
 (34, 0.0),
 (35, 0.015012432018092483),
 (36, 0.013527399267592246),
 (37, 0.0214

In [22]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similar_movies

[(20, 1.0),
 (38, 0.4249628989247758),
 (511, 0.1546628258528742),
 (234, 0.14543978900641946),
 (46, 0.13409793491936695),
 (4427, 0.12790908746010515),
 (126, 0.12406910285496274),
 (1004, 0.1238253687292483),
 (631, 0.1219930618822603),
 (9, 0.1219297197648043),
 (1191, 0.12073066243791428),
 (1760, 0.1207251224466345),
 (788, 0.12063556601439618),
 (79, 0.11947157308924512),
 (7, 0.11828290690100095),
 (33, 0.11754797974032946),
 (30, 0.11460131343801569),
 (182, 0.11260355641455022),
 (16, 0.11215733913329524),
 (1051, 0.11077198593157572),
 (203, 0.11046744828834201),
 (4759, 0.11010407896799432),
 (64, 0.10973548154912748),
 (471, 0.10953594698723658),
 (305, 0.10849087116405329),
 (3199, 0.10556393826572853),
 (174, 0.1052288002656489),
 (448, 0.10494339689569739),
 (4019, 0.10411822473090207),
 (1120, 0.10342666981010075),
 (784, 0.10222805977049187),
 (348, 0.10150273727278174),
 (2886, 0.10109014291908276),
 (2220, 0.1004412125948663),
 (3571, 0.1004269436073222),
 (970, 0.1

In [23]:
print('Here are the movies suggested for you: ')
i = 1
for mov in sorted_similar_movies:
    index = mov[0]
    index_title = movie[movie.index == index]['title'].values[0]
    if(i<21):
        print(i, ' ', index_title)
        i+=1

Here are the movies suggested for you: 
1   The Amazing Spider-Man
2   The Amazing Spider-Man 2
3   X-Men
4   The Croods
5   X-Men: Days of Future Past
6   Charly
7   Thor: The Dark World
8   The Boat That Rocked
9   Ice Age
10   Batman v Superman: Dawn of Justice
11   Small Soldiers
12   Anonymous
13   Deadpool
14   Iron Man 2
15   Avengers: Age of Ultron
16   X-Men: The Last Stand
17   Spider-Man 2
18   Ant-Man
19   The Avengers
20   Prisoners


In [24]:
movie_name = input('Enter your favourite movie name: ')
titles = movie['title'].to_list()
close_match = difflib.get_close_matches(movie_name, titles)
close_match = close_match[0]
movie_index = movie[movie.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[movie_index]))
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print('Here are the movies suggested for you: ')
i = 1
for mov in sorted_similar_movies:
    index = mov[0]
    index_title = movie[movie.index == index]['title'].values[0]
    if(i<21):
        print(i, ' ', index_title)
        i+=1

Enter your favourite movie name: avengers
Here are the movies suggested for you: 
1   The Avengers
2   Avengers: Age of Ultron
3   Captain America: The Winter Soldier
4   Captain America: Civil War
5   Iron Man 2
6   Thor: The Dark World
7   X-Men
8   The Incredible Hulk
9   X-Men: Apocalypse
10   Ant-Man
11   Thor
12   X2
13   X-Men: The Last Stand
14   Deadpool
15   X-Men: Days of Future Past
16   Captain America: The First Avenger
17   The Amazing Spider-Man 2
18   The Image Revolution
19   Iron Man
20   Iron Man 3
